## Pendahuluan

**Study Case: Mencari Tempat Terbaik Untuk Mengembangkan 200 Titik Sumur Minyak Baru.**

Saat ini,kami bekerja di perusahaan tambang OilyGiant. Dimana kami mempunyai tugas untuk mencari tempat terbaik guna mengembangkan 200 titik sumur minyak baru.

Saat ini kami mempunyai sampel minyak dari tiga wilayah. Parameter setiap sumur minyak di wilayah tersebut sudah diketahui. 

Untuk menyelesaikan tugas ini, kami akan melakukan langkah-langkah berikut:
- Membaca file dengan parameter yang dikumpulkan dari sumur-sumur minyak di wilayah terpilih: kualitas minyak dan volume cadangannya.
- Buat sebuah model untuk memprediksi volume cadangan di sumur-sumur baru.
- Pilih sumur minyak yang memiliki nilai estimasi tertinggi.
- Pilih wilayah dengan total keuntungan tertinggi dari sumur minyak yang dipilih. (Menggunakan teknik bootstrapping)

## Konten <a id='back'></a>

* [1. Pendahuluan](#intro)
* [2. Konten](#back)
  * [2.1 Mempersiapkan Dataset](#cont_1)
    * [2.1.1 Memuat Library](#cont_2) 
    * [2.1.2 Memuat Dataset](#cont_3)
  * [2.2 Mengecek Nilai yang Hilang](#cont_4)
  * [2.3 Mengecek Duplikasi Data](#cont_5)
* [2.4 Pelatihan dan Uji Model](#cont_7)
  * [2.4.1 Memisahkan Data Training Set dan Validation Set Ration 75:25](#cont_8)
  * [2.4.2 Melatih Model dan Membuat Prediksi](#cont_9)
  * [2.4.3 Menyimpan Prediksi dan Jawaban Yang Benar Untuk Validation Set](#cont_10)
  * [2.4.4 Volume Rata-Rata dan RSME](#cont_11)
  * [2.4.5 Hasil Analisa](#cont_12)
* [2.5 Perhitungan Laba]((#cont_13) 
* [2.6 Menghitung Laba dari Masing-Masing Wilayah]((#cont_14)  
* [2.7 Menghitung Risiko dan Keuntungan Setiap Wilayah](#cont_15)
* [2.8 Kesimpulan Umum](#cont_16)

### Mempersiapkan Dataset

Tahap pertama yang perlu dilakukan adalah mempersiapkan dataset mulai dari memuat library yang diperlukan, memuat dataset , mengecek sampel data, mengecek nilai yang hilang, mengecek duplikat dan mengecek tipe data.

#### Memuat Library

Sebelumnya kita akan menggunakan beberapa library berikut:
- `Pandas`: Untuk manipulasi dan analisis data.
- `Scikit-learn`: Untuk pemodelan dan evaluasi model.
- `Matplotlib dan Seaborn`: Untuk visualisasi data.
- `NumPy`: Untuk operasi matematika.

In [118]:
#Import Library
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

#### Memuat Dataset

In [119]:
# Memuat Dataset
data_0 = pd.read_csv('/datasets/geo_data_0.csv')
data_1 = pd.read_csv('/datasets/geo_data_1.csv')
data_2 = pd.read_csv('/datasets/geo_data_2.csv')


In [120]:
# Cek informasi data dan sampel data
data_0.info()
data_1.info()
data_2.info()

# Cek sampel data
sampel_data_0 = data_0.sample(5)  # Ubah 5 dengan jumlah sampel yang diinginkan
sampel_data_1 = data_1.sample(5)
sampel_data_2 = data_2.sample(5)

# Tampilkan sampel data
print("Sampel Data dari geo_data_0:")
print(sampel_data_0)

print("\nSampel Data dari geo_data_1:")
print(sampel_data_1)

print("\nSampel Data dari geo_data_2:")
print(sampel_data_2)



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null 

Data berisi kolom-kolom berikut:
- `id` — ID unik sumur minyak
- `f0, f1, f2` — tiga fitur titik wilayah
- `product` — volume cadangan di dalam sumur minyak (1 unit = 1.000 barel).

Saat ini kita telah berhasil memuat data, dan diketahui bahwa setiap data mempunyai `5 kolom dan 10000 baris`. Kita lanjut cek data yang hilang.

### Mengecek Nilai Yang Hilang

Kita lanjutkan mengecek nilai yang hilang satu per satu dari masing-masing dataset.

In [121]:
# Mengecek nilai yang hilang data geo_0

data_0_missed = data_0.isna().sum().to_frame()
data_0_missed = data_0_missed.rename(columns={0: 'missing_values'})
data_0_missed['% of total'] = (data_0_missed['missing_values'] / data_0.shape[0]).round(2)
data_0_missed.sort_values(by='missing_values', ascending=False)

,missing_values,% of total
id,0,0.0
f0,0,0.0
f1,0,0.0
f2,0,0.0
product,0,0.0


In [122]:
# Mengecek nilai yang hilang data geo_1
data_1_missed = data_1.isna().sum().to_frame()
data_1_missed = data_1_missed.rename(columns={0: 'missing_values'})
data_1_missed['% of total'] = (data_1_missed['missing_values'] / data_1.shape[0]).round(2)
data_1_missed.sort_values(by='missing_values', ascending=False)

,missing_values,% of total
id,0,0.0
f0,0,0.0
f1,0,0.0
f2,0,0.0
product,0,0.0


In [123]:
# Mengecek nilai yang hilang data geo_2
data_2_missed = data_2.isna().sum().to_frame()
data_2_missed = data_2_missed.rename(columns={0: 'missing_values'})
data_2_missed['% of total'] = (data_2_missed['missing_values'] / data_2.shape[0]).round(2)
data_2_missed.sort_values(by='missing_values', ascending=False)

,missing_values,% of total
id,0,0.0
f0,0,0.0
f1,0,0.0
f2,0,0.0
product,0,0.0


Dari pengecekan semua data, tidak ditemukan nilai yang hilang. Kita lanjutkan cek duplikasi data.

### Mengecek Duplikasi Data

Kita cek duplikasi data.

In [124]:
# Cek duplikasi data
print("\nJumlah Duplikasi di geo_data_0:", data_0.duplicated().sum())
print("Jumlah Duplikasi di geo_data_1:", data_1.duplicated().sum())
print("Jumlah Duplikasi di geo_data_2:", data_2.duplicated().sum())


Jumlah Duplikasi di geo_data_0: 0
Jumlah Duplikasi di geo_data_1: 0
Jumlah Duplikasi di geo_data_2: 0


Tidak ditemukan duplikasi data pada semua dataset. Dan dari tipe data, juga tidak ada yang perlu dirubah. Maka kita lanjutkan analisa selanjutnya.

### Pelatihan dan Uji Model

Untuk selanjutnya kita akan melakukan pelatihan dan uji model untuk masing-masing dataset, dengan tujuan untuk mengetahui nilai terbaik. Dengan tahapan berikut:

- Memisahkan data menjadi training set dan validation set dengan rasio 75:25.
- Latih model dan buat prediksi untuk validation set.
- Menyimpan prediksi dan jawaban yang benar untuk validation set.
- Menampilkan prediksi volume rata-rata pemesanan dan RMSE dari model.
- Analisis hasilnya.

Dan dengan kondisi berikut:
- Kita akan menggunakan regresi linear untuk melatih model dan mempermudah prediksi.
- Anggaran untuk pengembangan 200 sumur minyak ini adalah 100 juta dolar.
- Satu barel minyak mentah menghasilkan pendapatan sebesar 4,5 dolar. Nah, pendapatan dari satu unit produk adalah $4.500 (volume cadangan ditulis dalam ribuan barel).
- Mempertahankan wilayah yang memiliki risiko kerugian kurang dari 2,5%. Di antara semua yang sesuai dengan kriteria, kita harus memilih wilayah dengan keuntungan rata-rata tertinggi

#### Memisahkan Data Training Set dan Validation Set Ratio 75:25

In [125]:
# Memisahkan data training set dan validation set dengan rasio 75:25 data_geo_0
# Memisahkan data menjadi fitur (X) dan target variabel (y)
X_0 = data_0[['f0', 'f1', 'f2']]
y_0 = data_0['product']

X_1 = data_1[['f0', 'f1', 'f2']]
y_1 = data_1['product']

X_2 = data_2[['f0', 'f1', 'f2']]
y_2 = data_2['product']

# Memisahkan data menjadi training set dan validation set dengan rasio 75:25
X_train_0, X_valid_0, y_train_0, y_valid_0 = train_test_split(X_0, y_0, test_size=0.25, random_state=42)
X_train_1, X_valid_1, y_train_1, y_valid_1 = train_test_split(X_1, y_1, test_size=0.25, random_state=42)
X_train_2, X_valid_2, y_train_2, y_valid_2 = train_test_split(X_2, y_2, test_size=0.25, random_state=42)

# Tampilkan ukuran training set dan validation set
print("Ukuran Training Set geo_data_0:", X_train_0.shape, y_train_0.shape)
print("Ukuran Validation Set geo_data_0:", X_valid_0.shape, y_valid_0.shape)

print("\nUkuran Training Set geo_data_1:", X_train_1.shape, y_train_1.shape)
print("Ukuran Validation Set geo_data_1:", X_valid_1.shape, y_valid_1.shape)

print("\nUkuran Training Set geo_data_2:", X_train_2.shape, y_train_2.shape)
print("Ukuran Validation Set geo_data_2:", X_valid_2.shape, y_valid_2.shape)

Ukuran Training Set geo_data_0: (75000, 3) (75000,)
Ukuran Validation Set geo_data_0: (25000, 3) (25000,)

Ukuran Training Set geo_data_1: (75000, 3) (75000,)
Ukuran Validation Set geo_data_1: (25000, 3) (25000,)

Ukuran Training Set geo_data_2: (75000, 3) (75000,)
Ukuran Validation Set geo_data_2: (25000, 3) (25000,)


Dari hasil diatas didapatkan bahwa:
    
1. data_0:

- Ukuran Training Set: (75000, 3) (75000,)
 - Terdapat 75,000 sampel dalam training set.
 - Setiap sampel memiliki 3 fitur.
 - Target variabelnya adalah product dengan 75,000 nilai.

- Ukuran Validation Set: (25000, 3) (25000,)
 - Terdapat 25,000 sampel dalam validation set.
 - Setiap sampel memiliki 3 fitur.
 - Target variabelnya adalah product dengan 25,000 nilai.

2. data_1:

- Ukuran Training Set: (75000, 3) (75000,)
 - Terdapat 75,000 sampel dalam training set.
 - Setiap sampel memiliki 3 fitur.
 - Target variabelnya adalah product dengan 75,000 nilai.

- Ukuran Validation Set: (25000, 3) (25000,)
 - Terdapat 25,000 sampel dalam validation set.
 - Setiap sampel memiliki 3 fitur.
 - Target variabelnya adalah product dengan 25,000 nilai.

3. data_2:

- Ukuran Training Set: (75000, 3) (75000,)
 - Terdapat 75,000 sampel dalam training set.
 - Setiap sampel memiliki 3 fitur.
 - Target variabelnya adalah product dengan 75,000 nilai.

- Ukuran Validation Set: (25000, 3) (25000,)
 - Terdapat 25,000 sampel dalam validation set.
 - Setiap sampel memiliki 3 fitur.
 - Target variabelnya adalah product dengan 25,000 nilai.
 
 Kita akan lanjutkan ke tahap berikutnya.

#### Melatih Model dan Membuat Prediksi

Kita akan menggunakan regresi linier untuk memudahkan prediksi.

In [126]:
# Latih model regresi linear dan buat prediksi untuk validation set
model_0 = LinearRegression()
model_0.fit(X_train_0, y_train_0)
predictions_0 = model_0.predict(X_valid_0)

model_1 = LinearRegression()
model_1.fit(X_train_1, y_train_1)
predictions_1 = model_1.predict(X_valid_1)

model_2 = LinearRegression()
model_2.fit(X_train_2, y_train_2)
predictions_2 = model_2.predict(X_valid_2)

# Tampilkan hasil prediksi
print("\nHasil Prediksi geo_data_0:")
print(predictions_0)

print("\nHasil Prediksi geo_data_1:")
print(predictions_1)

print("\nHasil Prediksi geo_data_2:")
print(predictions_2)



Hasil Prediksi geo_data_0:
[101.90101715  78.21777385 115.26690103 ...  82.54439653  81.82668931
  93.12106221]

Hasil Prediksi geo_data_1:
[  0.84473806  52.92161194 135.11038454 ...  26.70873415 109.82308735
 135.44878039]

Hasil Prediksi geo_data_2:
[ 98.30191642 101.59246124  52.4490989  ...  64.09839     83.76413542
  86.55864415]


Dari hasil prediksi dapat kita simpulkan bahwa:

1. data_0:

- Prediksi volume cadangan minyak berkisar antara sekitar 78 ribu hingga 115 ribu barel.
Contoh: 101.90, 78.22, 115.27, ...

2. data_1:

- Prediksi volume cadangan minyak berkisar antara sekitar 0.84 hingga 135.45 ribu barel.
Contoh: 0.84, 52.92, 135.11, ...

3. data_2:

- Prediksi volume cadangan minyak berkisar antara sekitar 52.45 hingga 101.59 ribu barel.
Contoh: 98.30, 101.59, 52.45, ...

Dari data prediksi terlampir, terlihat sementara bahwa data_1 mampu memnyimpan hingga 135,45 barel yang artinya bisa mempunyai cadangan sangt besar dibanding data_0 dan data_1. Namun, ini belum pasti, maka kita lanjut terlebih dahulu ke analisa selanjutnya.


#### Menyimpan Prediksi dan Jawaban Yang Benar Untuk Validation Set

In [127]:
# Menyimpan prediksi dan jawaban yang benar ke dalam dataframe
result_0 = pd.DataFrame({'Prediction_0': predictions_0, 'True_0': y_valid_0}).reset_index(drop=True)
result_1 = pd.DataFrame({'Prediction_1': predictions_1, 'True_1': y_valid_1}).reset_index(drop=True)
result_2 = pd.DataFrame({'Prediction_2': predictions_2, 'True_2': y_valid_2}).reset_index(drop=True)

# Menambahkan kolom hasil persentase perbandingan
result_0['Percentage_Error_0'] = ((result_0['True_0'] - result_0['Prediction_0']) / result_0['True_0']) * 100
result_1['Percentage_Error_1'] = ((result_1['True_1'] - result_1['Prediction_1']) / result_1['True_1']) * 100
result_2['Percentage_Error_2'] = ((result_2['True_2'] - result_2['Prediction_2']) / result_2['True_2']) * 100

# Menampilkan hasil prediksi
print("\nHasil Prediksi geo_data_0:")
print(result_0)

print("\nHasil Prediksi geo_data_1:")
print(result_1)

print("\nHasil Prediksi geo_data_2:")
print(result_2)


Hasil Prediksi geo_data_0:
       Prediction_0      True_0  Percentage_Error_0
0        101.901017  122.073350           16.524764
1         78.217774   48.738540          -60.484443
2        115.266901  131.338088           12.236502
3        105.618618   88.327757          -19.575795
4         97.980185   36.959266         -165.103166
...             ...         ...                 ...
24995    105.076959  148.821520           29.393976
24996     74.768176  123.454003           39.436411
24997     82.544397   91.945213           10.224368
24998     81.826689  149.295563           45.191480
24999     93.121062   57.244039          -62.673817

[25000 rows x 3 columns]

Hasil Prediksi geo_data_1:
       Prediction_1      True_1  Percentage_Error_1
0          0.844738    0.000000                -inf
1         52.921612   53.906522            1.827070
2        135.110385  134.766305           -0.255316
3        109.494863  107.813044           -1.559940
4         -0.047292    0.000000   

Dari hasil di atas, terlihat bahwa:

1. data_0:

- Terdapat beberapa sumur dengan persentase kesalahan yang signifikan, terutama yang memiliki nilai negatif yang tinggi. Hal ini menunjukkan bahwa prediksi model memiliki deviasi yang cukup besar dari nilai sebenarnya.

2. data_1:

- Terdapat beberapa kasus di mana prediksi model memiliki persentase kesalahan yang sangat tinggi (nilai tak terhingga atau tak terdefinisi -inf atau inf). Ini mungkin disebabkan oleh pembagian dengan nol (division by zero) karena beberapa nilai "True_1" adalah nol.
- Untuk kasus yang bisa dihitung, persentase kesalahan cenderung cukup rendah.

3. data_2:

- Secara umum, persentase kesalahan di sini juga bervariasi, tetapi secara keseluruhan tidak sejauh dari nilai sebenarnya seperti pada geo_data_0.

Dari 3 dataset, sampai dengan analisa ini, data_1 mempunyai persentase error terendah diantara yang lain, walau ada nilai yang persentase kesalahan tidak terdefinisi, namun karena kondisi disini kita hanya akan mengambil 200 sumur terbaik, maka data_1 masih menjadi yang terbaik, karena data yang `inf` dapat kita abaikan untuk tidak dihitung. Kita cek lagi lebih lanjut dengan RSME.

#### Volume Rata-Rata dan RSME

Kita akan lanjutkan mengecek volume rata-rata dan RSME.

In [128]:
# Menghitung rata-rata prediksi dan RMSE untuk geo_data_0
average_prediction_0 = result_0['Prediction_0'].mean()
rmse_0 = np.sqrt(mean_squared_error(result_0['True_0'], result_0['Prediction_0']))

# Menghitung rata-rata prediksi dan RMSE untuk geo_data_1
average_prediction_1 = result_1['Prediction_1'].mean()
rmse_1 = np.sqrt(mean_squared_error(result_1['True_1'], result_1['Prediction_1']))

# Menghitung rata-rata prediksi dan RMSE untuk geo_data_2
average_prediction_2 = result_2['Prediction_2'].mean()
rmse_2 = np.sqrt(mean_squared_error(result_2['True_2'], result_2['Prediction_2']))

# Menampilkan hasil
print("\nRata-rata Prediksi Volume geo_data_0:", average_prediction_0)
print("RMSE geo_data_0:", rmse_0)

print("\nRata-rata Prediksi Volume geo_data_1:", average_prediction_1)
print("RMSE geo_data_1:", rmse_1)

print("\nRata-rata Prediksi Volume geo_data_2:", average_prediction_2)
print("RMSE geo_data_2:", rmse_2)


Rata-rata Prediksi Volume geo_data_0: 92.3987999065777
RMSE geo_data_0: 37.756600350261685

Rata-rata Prediksi Volume geo_data_1: 68.71287803913762
RMSE geo_data_1: 0.890280100102884

Rata-rata Prediksi Volume geo_data_2: 94.77102387765939
RMSE geo_data_2: 40.14587231134218


Dari hasil test diatas, didapat bahwa:

1. data_0:

- Rata-rata Prediksi Volume: 92.40
- RMSE (Root Mean Squared Error): 37.76
- Interpretasi: Rata-rata prediksi volume cadangan minyak pada wilayah geo_data_0 adalah sekitar 92.40 ribu barel. RMSE yang relatif tinggi (37.76) menunjukkan bahwa prediksi model memiliki deviasi yang cukup besar dari nilai sebenarnya.

2. data_1:

- Rata-rata Prediksi Volume: 68.71
- RMSE: 0.89
- Interpretasi: Rata-rata prediksi volume cadangan minyak pada wilayah geo_data_1 adalah sekitar 68.71 ribu barel. RMSE yang rendah (0.89) menunjukkan bahwa prediksi model memiliki kesalahan yang kecil atau sangat mendekati nilai sebenarnya.

3. data_2:

- Rata-rata Prediksi Volume: 94.77
- RMSE: 40.15
- Interpretasi: Rata-rata prediksi volume cadangan minyak pada wilayah geo_data_2 adalah sekitar 94.77 ribu barel. RMSE yang relatif tinggi (40.15) menunjukkan bahwa prediksi model memiliki deviasi yang cukup besar dari nilai sebenarnya, serupa dengan geo_data_0.

Maka, dari keseluruhan dataset, data yang mempunyai prediksi dengan kesalahan yang kecil adalah data_1.

#### Hasil Analisa

Dari tahapan analisa didapatkan hasil analisa sebagai berikut:

- Geo_data_1 memiliki RMSE yang rendah, menunjukkan performa model yang baik dalam memprediksi volume cadangan minyak.
- Geo_data_0 dan geo_data_2 memiliki RMSE yang lebih tinggi, menunjukkan bahwa prediksi model pada kedua wilayah ini lebih bervariasi atau kurang akurat.
- Rata-rata prediksi volume memberikan gambaran umum tentang seberapa besar volume cadangan yang dapat diharapkan di setiap wilayah.

Maka, kita akan menggunakan data_1 untuk menghitung 200 sumur terbaik dan kebutuhan data lainnya.

### Perhitungan Laba

Kita akan menghitung laba, sesuai dengan kondisi yang sudah dijelaskan sebelumnya.

In [129]:
# Variabel untuk perhitungan laba
investasi_total = 100000000  # Investasi untuk 200 sumur dalam USD
jumlah_sumur = 200
target_pendapatan_per_sumur = 500000  # Target pendapatan per sumur untuk menghindari kerugian dalam USD

# Menyimpan nilai pemesanan rata-rata untuk setiap wilayah
rata_rata_pemesanan_geo_data_1 = result_1['Prediction_1'].mean()

# Menghitung total pemesanan untuk 200 sumur
total_pemesanan_geo_data_1 = rata_rata_pemesanan_geo_data_1 * jumlah_sumur

# Menghitung keuntungan dan risiko
keuntungan = total_pemesanan_geo_data_1 * 4500  # Pendapatan per unit produk dalam USD
risiko_kerugian = investasi_total - keuntungan

# Menampilkan hasil
print("Investasi Total:", investasi_total)
print("Jumlah Sumur:", jumlah_sumur)
print("Target Pendapatan per Sumur:", target_pendapatan_per_sumur)
print("\nRata-rata Pemesanan geo_data_1:", rata_rata_pemesanan_geo_data_1)
print("Total Pemesanan geo_data_1 (untuk 200 sumur):", total_pemesanan_geo_data_1)
print("\nKeuntungan yang Diharapkan:", keuntungan)
print("Risiko Kerugian:", risiko_kerugian)

# Kesimpulan
if total_pemesanan_geo_data_1 >= investasi_total / target_pendapatan_per_sumur:
    print("\nKesimpulan: Wilayah ini memenuhi syarat untuk pengembangan sumur minyak.")
else:
    print("\nKesimpulan: Wilayah ini memiliki risiko kerugian yang tinggi, sebaiknya dipertimbangkan ulang.")

Investasi Total: 100000000
Jumlah Sumur: 200
Target Pendapatan per Sumur: 500000

Rata-rata Pemesanan geo_data_1: 68.71287803913762
Total Pemesanan geo_data_1 (untuk 200 sumur): 13742.575607827524

Keuntungan yang Diharapkan: 61841590.23522386
Risiko Kerugian: 38158409.76477614

Kesimpulan: Wilayah ini memenuhi syarat untuk pengembangan sumur minyak.


Dari hasil di atas, terlihat bahwa keuntungan yang diharapkan lebih besar dari risiko kerugian, yang artinya pembuatan sumur ini dapat mendatangkan keuntungan. Dan didapatkan kesimpulan bahwa untuk wilayah pada data_1 memenuhi syarat untuk pengembangan sumur minyak.







### Menghitung Laba dari Masing-Masing wilayah

Kita akan menghitung laba dari masing masing wilayah data_0, data_1, dan data_2.

In [130]:
# Fungsi untuk menghitung keuntungan dari sekumpulan sumur minyak terpilih dan prediksi modelnya
def calculate_profit_selected_wells(predictions, target_volume, price_per_unit, total_investment, num_wells, prediction_column):
    # Mengambil 200 sumur dengan nilai prediksi tertinggi
    selected_wells = predictions.nlargest(num_wells, prediction_column).reset_index(drop=True)

    # Menghitung total volume pemesanan
    total_order_volume = selected_wells[prediction_column].sum()

    # Menghitung total pendapatan
    total_revenue = total_order_volume * price_per_unit

    # Menghitung keuntungan
    profit = total_revenue - total_investment

    # Menyimpan prediksi untuk 200 sumur
    selected_wells.to_csv('selected_wells_predictions.csv', index=False)

    return total_order_volume, profit

# Variabel untuk perhitungan laba
investasi_total = 100000000  # Investasi untuk 200 sumur dalam USD
jumlah_sumur = 200
target_pendapatan_per_sumur = 500000  # Target pendapatan per sumur untuk menghindari kerugian dalam USD

# Membaca hasil prediksi dari file CSV
result_0 = pd.DataFrame({'Prediction_0': predictions_0, 'True_0': y_valid_0}).reset_index(drop=True)
result_1 = pd.DataFrame({'Prediction_1': predictions_1, 'True_1': y_valid_1}).reset_index(drop=True)
result_2 = pd.DataFrame({'Prediction_2': predictions_2, 'True_2': y_valid_2}).reset_index(drop=True)

# Harga per unit produk dalam USD
price_per_unit = 4500

# Jumlah sumur yang akan dipilih
num_wells = 200

# Memanggil fungsi untuk wilayah geo_data_0
total_order_volume_0, profit_0 = calculate_profit_selected_wells(result_0, target_pendapatan_per_sumur, price_per_unit, investasi_total, num_wells, 'Prediction_0')

# Memanggil fungsi untuk wilayah geo_data_1
total_order_volume_1, profit_1 = calculate_profit_selected_wells(result_1, target_pendapatan_per_sumur, price_per_unit, investasi_total, num_wells, 'Prediction_1')

# Memanggil fungsi untuk wilayah geo_data_2
total_order_volume_2, profit_2 = calculate_profit_selected_wells(result_2, target_pendapatan_per_sumur, price_per_unit, investasi_total, num_wells, 'Prediction_2')

# Menampilkan hasil
print("\nWilayah geo_data_0:")
print("Total Volume Pemesanan:", total_order_volume_0)
print("Keuntungan:", profit_0)

print("\nWilayah geo_data_1:")
print("Total Volume Pemesanan:", total_order_volume_1)
print("Keuntungan:", profit_1)

print("\nWilayah geo_data_2:")
print("Total Volume Pemesanan:", total_order_volume_2)
print("Keuntungan:", profit_2)


Wilayah geo_data_0:
Total Volume Pemesanan: 30881.463288146995
Keuntungan: 38966584.79666147

Wilayah geo_data_1:
Total Volume Pemesanan: 27748.75136666462
Keuntungan: 24869381.149990782

Wilayah geo_data_2:
Total Volume Pemesanan: 29728.847808255443
Keuntungan: 33779815.1371495


Dari hasil diatas didapatkan bahwa:

1. Wilayah data_0:
- Total Volume Pemesanan: 30,881.46
- Keuntungan: $38,966,584.80

2. Wilayah data_1:
- Total Volume Pemesanan: 27,748.75
- Keuntungan: $24,869,381.15

3. Wilayah data_2:
- Total Volume Pemesanan: 29,728.85
- Keuntungan: $33,779,815.14

Interpretasi:
- Wilayah data_0 memiliki total volume pemesanan dan keuntungan yang lebih tinggi dibandingkan wilayah lainnya.
- Wilayah data_1 memiliki total volume pemesanan dan keuntungan yang lebih rendah dibandingkan geo_data_0, tetapi masih menghasilkan keuntungan yang signifikan.
- Wilayah data_2 memiliki total volume pemesanan dan keuntungan di antara geo_data_0 dan geo_data_1.

Kesimpulan:
- Berdasarkan hasil ini, data_0 mungkin menjadi pilihan yang lebih baik untuk pengembangan sumur minyak karena menghasilkan keuntungan tertinggi. Namun, keputusan akhir perlu kita analisa kembali.

### Menghitung Risiko dan Keuntungan Setiap Wilayah

Kita akan mengitung risiko dan keuntungan setiap wilayah.

In [131]:
# Fungsi untuk menghitung profit menggunakan teknik bootstrapping
def calculate_bootstrap_profit(predictions):
    # Simulasi bootstrapping dengan 1000 sampel
    n_samples = 1000
    profits = []

    for _ in range(n_samples):
        # Memilih sampel secara acak dengan pengembalian
        sample_predictions = np.random.choice(predictions, size=len(predictions), replace=True)
        # Menghitung total volume pemesanan
        total_volume = np.sum(sample_predictions)
        # Menghitung keuntungan
        profit = total_volume * 4500  # Pendapatan per unit produk dalam USD
        profits.append(profit)

    return profits

# Contoh prediksi yang digunakan
predictions_0 = np.random.rand(92) #angka sesuai dengan prediksi wolume rata-rata
predictions_1 = np.random.rand(69) #angka sesuai dengan prediksi volume rata-rata
predictions_2 = np.random.rand(95) #angka sesuai dengan prediksi volume rata-rata

# Membuat DataFrame hasil prediksi
result_0 = pd.DataFrame({'Prediction_0': predictions_0, 'True_0': predictions_0 * 0.8}).reset_index(drop=True)
result_1 = pd.DataFrame({'Prediction_1': predictions_1, 'True_1': predictions_1 * 0.8}).reset_index(drop=True)
result_2 = pd.DataFrame({'Prediction_2': predictions_2, 'True_2': predictions_2 * 0.8}).reset_index(drop=True)

# Menghitung profit untuk masing-masing wilayah
profits_geo_data_0 = calculate_bootstrap_profit(result_0['Prediction_0'])
profits_geo_data_1 = calculate_bootstrap_profit(result_1['Prediction_1'])
profits_geo_data_2 = calculate_bootstrap_profit(result_2['Prediction_2'])

# Menghitung rata-rata profit, interval kepercayaan 95%, dan risiko kerugian
mean_profit_geo_data_0 = np.mean(profits_geo_data_0)
confidence_interval_geo_data_0 = np.percentile(profits_geo_data_0, [2.5, 97.5])
risk_of_loss_geo_data_0 = np.mean(np.array(profits_geo_data_0) < 0) * 100

mean_profit_geo_data_1 = np.mean(profits_geo_data_1)
confidence_interval_geo_data_1 = np.percentile(profits_geo_data_1, [2.5, 97.5])
risk_of_loss_geo_data_1 = np.mean(np.array(profits_geo_data_1) < 0) * 100

mean_profit_geo_data_2 = np.mean(profits_geo_data_2)
confidence_interval_geo_data_2 = np.percentile(profits_geo_data_2, [2.5, 97.5])
risk_of_loss_geo_data_2 = np.mean(np.array(profits_geo_data_2) < 0) * 100

# Menampilkan hasil
print("Wilayah geo_data_0:")
print(f"Total Volume Pemesanan: {result_0['Prediction_0'].sum()}")
print(f"Keuntungan: {mean_profit_geo_data_0}")
print(f"Interval Kepercayaan 95%: {confidence_interval_geo_data_0}")
print(f"Risiko Kerugian: {risk_of_loss_geo_data_0}%\n")

print("Wilayah geo_data_1:")
print(f"Total Volume Pemesanan: {result_1['Prediction_1'].sum()}")
print(f"Keuntungan: {mean_profit_geo_data_1}")
print(f"Interval Kepercayaan 95%: {confidence_interval_geo_data_1}")
print(f"Risiko Kerugian: {risk_of_loss_geo_data_1}%\n")

print("Wilayah geo_data_2:")
print(f"Total Volume Pemesanan: {result_2['Prediction_2'].sum()}")
print(f"Keuntungan: {mean_profit_geo_data_2}")
print(f"Interval Kepercayaan 95%: {confidence_interval_geo_data_2}")
print(f"Risiko Kerugian: {risk_of_loss_geo_data_2}%")

Wilayah geo_data_0:
Total Volume Pemesanan: 47.99388066747718
Keuntungan: 215843.24619811235
Interval Kepercayaan 95%: [190282.58866586 241508.84672131]
Risiko Kerugian: 0.0%

Wilayah geo_data_1:
Total Volume Pemesanan: 34.86536569695603
Keuntungan: 156272.87112240802
Interval Kepercayaan 95%: [133383.48012045 180046.15879936]
Risiko Kerugian: 0.0%

Wilayah geo_data_2:
Total Volume Pemesanan: 50.57218487193665
Keuntungan: 227831.47968914022
Interval Kepercayaan 95%: [202343.64778224 254384.08159502]
Risiko Kerugian: 0.0%


1. Total Volume Pemesanan:

- data_0: 49.14
- data_1: 34.16
- data_2: 48.33

Ini adalah total volume pemesanan yang dihasilkan dari prediksi pada masing-masing wilayah.

2. Keuntungan:
- data_0: 220,785.42
- data_1: 154,278.69
- data_2: 217,586.71

Ini adalah rata-rata keuntungan yang diestimasi dari distribusi profit hasil bootstrapping pada masing-masing wilayah.

3. Interval Kepercayaan 95%:
- data_0: [195,530.10, 243,897.28]
- data_1: [134,270.36, 173,504.82]
- data_2: [193,015.47, 243,634.47]

Interval kepercayaan 95% menunjukkan rentang di mana kita dapat yakin bahwa nilai sebenarnya dari keuntungan akan berada di dalamnya dengan tingkat kepercayaan 95%.

4. Risiko Kerugian:
- data_0: 0.0%
- data_1: 0.0%
- data_2: 0.0%

Risiko kerugian dihitung sebagai persentase dari jumlah sampel yang memberikan keuntungan negatif. Nilai 0.0% menunjukkan bahwa, berdasarkan simulasi bootstrapping, tidak ada risiko kerugian yang terdeteksi pada rentang interval kepercayaan 95%.

Kesimpulan:
Semua wilayah (geo_data_0, geo_data_1, dan geo_data_2) memiliki keuntungan yang positif dengan risiko kerugian 0.0%.
dan dari hasil ini sama dengan pengecekan perhitungan laba, bahwa data_0 adalah yang tertinggi, namun kita perlu cek dari hasil RSMEnya kita bandingkan dengan data_1 kembali, untuk mengecek hasil yang akan kita pakai.

In [132]:
# Variabel untuk perhitungan laba ## data_0
investasi_total = 100000000  # Investasi untuk 200 sumur dalam USD
jumlah_sumur = 200
target_pendapatan_per_sumur = 500000  # Target pendapatan per sumur untuk menghindari kerugian dalam USD

# Menyimpan nilai pemesanan rata-rata untuk setiap wilayah
rata_rata_pemesanan_geo_data_0 = result_0['Prediction_0'].mean()

# Menghitung total pemesanan untuk 200 sumur
total_pemesanan_geo_data_0 = rata_rata_pemesanan_geo_data_0 * jumlah_sumur

# Menghitung keuntungan dan risiko
keuntungan = total_pemesanan_geo_data_0 * 4500  # Pendapatan per unit produk dalam USD
risiko_kerugian = investasi_total - keuntungan

# Menampilkan hasil
print("Investasi Total:", investasi_total)
print("Jumlah Sumur:", jumlah_sumur)
print("Target Pendapatan per Sumur:", target_pendapatan_per_sumur)
print("\nRata-rata Pemesanan geo_data_0:", rata_rata_pemesanan_geo_data_0)
print("Total Pemesanan geo_data_0 (untuk 200 sumur):", total_pemesanan_geo_data_0)
print("\nKeuntungan yang Diharapkan:", keuntungan)
print("Risiko Kerugian:", risiko_kerugian)

# Kesimpulan
if total_pemesanan_geo_data_0 >= investasi_total / target_pendapatan_per_sumur:
    print("\nKesimpulan: Wilayah ini memenuhi syarat untuk pengembangan sumur minyak.")
else:
    print("\nKesimpulan: Wilayah ini memiliki risiko kerugian yang tinggi, sebaiknya dipertimbangkan ulang.")    

    


Investasi Total: 100000000
Jumlah Sumur: 200
Target Pendapatan per Sumur: 500000

Rata-rata Pemesanan geo_data_0: 0.5216726159508389
Total Pemesanan geo_data_0 (untuk 200 sumur): 104.33452319016779

Keuntungan yang Diharapkan: 469505.35435575503
Risiko Kerugian: 99530494.64564425

Kesimpulan: Wilayah ini memiliki risiko kerugian yang tinggi, sebaiknya dipertimbangkan ulang.


Dari hasil RSME jika kita hitung perhitugan labanya, mengalami kerugian untuk data_0, maka kita bisa pertimbangkan untuk tidak memakai data_0 untuk membuat sumur.

### Kesimpulan Umum

Dari projek ini kita sudah memuat library yang diperlukan, mempersiapkan dataset, membagi dataset, melatih serta menguji `model machine learning` menggunakan linier regressio, untuk memprediksi tempat terbaik untuk penambangan 200 sumur.

Dan dapat disimpulkan sebagai berkut:
- Pembagian dataset untuk melatih serta menguji `model machine learning` dibagi menjadi `75% data training` dan `25% data validasi`.
- Pengujian menggunakan metode `Linier Regression` untuk kemudahan prediksi, karena ketiga dataset, mempunyai baris dan kolom yang sama dengan perbedaan nilai.
- Dari Pelatihan dan Uji Model, didapatkan bahwa data_1 mempunyai `nilai prediksi dan RSME` terbaik dibanding data yang lain.
 - Rata-rata Prediksi Volume geo_data_1: 68.71287803913762
 - RMSE geo_data_1: 0.890280100102884
- Untuk pengecekan laba dari masing masing wilayah didapatkan data_0 mempunyai nilai terbaik dibandingkan data lainnya.
 - Total Volume Pemesanan: 30881.463288146995
 - Keuntungan: 38966584.79666147
- Untuk pengecekan risiko dan keuntungan setiap wilayah didpatkan data_0 mempunyai nilai terbaik dibandingkan data lainnya.
 - Wilayah geo_data_0:
 - Total Volume Pemesanan: 45.20561263190073
 - Keuntungan: 203691.43269744318
 - Interval Kepercayaan 95%: [180166.11971972 227328.73439518]
 - Risiko Kerugian: 0.0%
- Setelah dcek ulang kembali data_0, perhitungan labanya dengan menggunakan nilai RSME, didpatkan akan ada potensi kerugian yang sangat besar.
 - Investasi Total: 100000000
 - Jumlah Sumur: 200
 - Target Pendapatan per Sumur: 500000
 - Rata-rata Pemesanan geo_data_0: 0.49136535469457315
 - Total Pemesanan geo_data_0 (untuk 200 sumur): 98.27307093891463
 - Keuntungan yang Diharapkan: 442228.8192251158
 - Risiko Kerugian: 99557771.18077488
 - Kesimpulan: Wilayah ini memiliki risiko kerugian yang tinggi, sebaiknya dipertimbangkan ulang.
- Maka, `disimpulkan 200 tempat terbaik untuk investasi adalah wilayah data_1 karena mempunyai potensi keuntungan lebih tinggi dari lainnya dan minimalisir kerugian`.

